# Matrix Factorization

We will experiment with the recent MovieLens 25M Dataset and build a recommender system using two approaches:
* Factorizing the user-item matrix using Spark ALS implementation
* Factorizing the item-item PMI maatrix using randomized SVD

In both settings we will index the item embeddings and inspect their quality using KNN queries.

# Part 1

### Download the dataset

In [0]:
!wget http://files.grouplens.org/datasets/movielens/ml-25m.zip
!unzip ml-25m
dbutils.fs.ls("file:/databricks/driver/ml-25m/")
dbutils.fs.mv("file:/databricks/driver/ml-25m/", "dbfs:/ml-25m/", recurse=True)

--2021-05-21 14:06:30-- http://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip 0%[ ] 0 --.-KB/s ml-25m.zip 0%[ ] 168.87K 828KB/s ml-25m.zip 0%[ ] 372.50K 911KB/s ml-25m.zip 0%[ ] 558.73K 910KB/s ml-25m.zip 0%[ ] 734.50K 897KB/s ml-25m.zip 0%[ ] 1.60M 1.57MB/s ml-25m.zip 3%[ ] 8.48M 6.91MB/s ml-25m.zip 6%[> ] 15.94M 11.1MB/s ml-25m.zip 9%[> ] 23.40M 14.3MB/s ml-25m.zip 12%[=> ] 30.85M 16.7MB/s ml-25m.zip 15%[==> ] 38.31M 18.7MB/s ml-25m.zip 18%[==> ] 45.77M 20.3MB/s ml-25m.zip 21%[===> ] 53.22M 21.6MB/s ml-25m.zip 24%[===> ] 60.68M 22.8MB/s ml-25m.zip 27%[====> ] 68.08M 23.8MB/s ml-25m.zip 30%[=====> ] 75.59M 24.6MB/s eta 7s ml-25m.zip 33%[=====> ] 83.05M 25.3MB/s eta 7s ml-25m.zip 36%[======> ] 90.51M 27.2MB/s eta 7s ml-25m.zip 39%[======> ] 97.97M 29.0MB/s eta 7s ml-25m.zip 42%[=======> ] 105.42M 30.8MB/s eta 7s ml-25m.zip 45%[========> ] 112.88M 34.2MB/s eta 5s ml-25m.zip 48%[========> ] 120.34M 35.9MB/s eta 5s ml-25m.zip 51%[=========> ] 127.79M 36.3MB/s eta 5s ml-25m.zip 54%[=========> ] 135.25M 36.3MB/s eta 5s ml-25m.zip 57%[==========> ] 142.71M 36.3MB/s eta 5s ml-25m.zip 60%[===========> ] 150.16M 36.3MB/s eta 3s ml-25m.zip 63%[===========> ] 157.62M 36.3MB/s eta 3s ml-25m.zip 66%[============> ] 165.08M 36.3MB/s eta 3s ml-25m.zip 69%[============> ] 172.54M 36.3MB/s eta 3s ml-25m.zip 72%[=============> ] 179.99M 36.3MB/s eta 3s ml-25m.zip 75%[==============> ] 187.45M 36.3MB/s eta 2s ml-25m.zip 78%[==============> ] 194.91M 36.3MB/s eta 2s ml-25m.zip 80%[===============> ] 202.36M 36.3MB/s eta 2s ml-25m.zip 83%[===============> ] 209.82M 36.3MB/s eta 2s ml-25m.zip 86%[================> ] 217.28M 36.3MB/s eta 2s ml-25m.zip 89%[================> ] 224.73M 36.3MB/s eta 1s ml-25m.zip 92%[=================> ] 232.19M 36.3MB/s eta 1s ml-25m.zip 95%[==================> ] 239.65M 36.3MB/s eta 1s ml-25m.zip 98%[==================> ] 247.10M 36.2MB/s eta 1s ml-25m.zip 100%[===================>] 249.84M 36.3MB/s in 7.9s 

2021-05-21 14:06:38 (31.8 MB/s) - ‘ml-25m.zip’ saved [261978986/261978986]

Archive: ml-25m.zip
 creating: ml-25m/
 inflating: ml-25m/tags.csv 
 inflating: ml-25m/links.csv 
 inflating: ml-25m/README.txt 
 inflating: ml-25m/ratings.csv 
 inflating: ml-25m/genome-tags.csv 
 inflating: ml-25m/genome-scores.csv 
 inflating: ml-25m/movies.csv 
Out[1]: True

In [0]:
dbutils.fs.ls("dbfs:/ml-25m/")

Out[2]: [FileInfo(path='dbfs:/ml-25m/README.txt', name='README.txt', size=10460),
 FileInfo(path='dbfs:/ml-25m/genome-scores.csv', name='genome-scores.csv', size=435164157),
 FileInfo(path='dbfs:/ml-25m/genome-tags.csv', name='genome-tags.csv', size=18103),
 FileInfo(path='dbfs:/ml-25m/links.csv', name='links.csv', size=1368578),
 FileInfo(path='dbfs:/ml-25m/movies.csv', name='movies.csv', size=3038099),
 FileInfo(path='dbfs:/ml-25m/ratings.csv', name='ratings.csv', size=678260987),
 FileInfo(path='dbfs:/ml-25m/tags.csv', name='tags.csv', size=38810332)]

### Loading the ratings dataset

In [0]:
df=spark.read.csv('dbfs:/ml-25m/movies.csv', header=True, inferSchema=True).cache()
df = spark.read.csv('dbfs:/ml-25m/ratings.csv', header=True, inferSchema=True)





In [0]:
df.head(5)

Out[25]: [Row(userId=1, movieId=296, rating=5.0, timestamp=1147880044),
 Row(userId=1, movieId=306, rating=3.5, timestamp=1147868817),
 Row(userId=1, movieId=307, rating=5.0, timestamp=1147868828),
 Row(userId=1, movieId=665, rating=5.0, timestamp=1147878820),
 Row(userId=1, movieId=899, rating=3.5, timestamp=1147868510)]

### Split the dataset
We want to randomly split the dataset into train and test parts

In [0]:
train, test = df.cache().randomSplit([0.8, 0.2], seed=12345)

### Build ALS model
Using the Spark ALS implementation described here https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html
Build a model using the ml-25m dataset.

How long does the training take, change the rank (i.e. the dimension of the vectors) from 10 to 20. How does that affect training speed ?

In [0]:
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

lines=df.rdd

In [0]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(train)

#

### Evaluation
Using the code described in the Spark documentation, evaluate how good your model is doing on the test set.
The goal is to predict the held out ratings.
A good metric could be RMSE or MAE.

In [0]:
#Evaluate the model by computing the RMSE on the test data
from pyspark.ml.evaluation import RegressionEvaluator

predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

Root-mean-square error = 0.811168443969297

### Inspecting the results

Retrieve the movie vectors from the learned model object (the property is called itemFactors).
and `collect` all these vectors in a list.

In [0]:
list_items_factors=model.itemFactors.collect()


[Row(id=10, features=[-0.5054094791412354, -0.16811057925224304, 0.029730208218097687, -0.24545277655124664, 0.3562713861465454, -0.11089249700307846, 0.07500549405813217, -0.17772066593170166, 0.16993831098079681, 0.05939692631363869]), Row(id=20, features=[-0.3835715055465698, -0.3141310214996338, -0.018945293501019478, 0.09248154610395432, 0.16460147500038147, 0.30024975538253784, 0.27581313252449036, -0.14986522495746613, 0.4188627004623413, -0.1695336103439331]), Row(id=30, features=[-0.5764872431755066, -0.1133156418800354, 0.18046694993972778, 0.4038027822971344, 0.5652745366096497, 0.24644571542739868, -0.19505378603935242, -0.13944993913173676, -0.03709063678979874, -0.33426016569137573]), Row(id=40, features=[-0.6072319149971008, -0.08431950956583023, 0.18808753788471222, 0.20475853979587555, 0.40244948863983154, 0.5176994800567627, 0.07542334496974945, -0.07959355413913727, -0.14364886283874512, -0.36823999881744385]), Row(id=50, features=[-0.1542559415102005, -0.5167540311813354, 0.3186124265193939, -0.06872913241386414, 0.4772151708602905, -0.03976709768176079, 0.1006397157907486, -0.22198398411273956, -0.16408243775367737, 0.10611006617546082]), Row(id=60, features=[-0.44351527094841003, -0.28377246856689453, -0.016714049503207207, 0.07434123754501343, -0.007914375513792038, 0.37936386466026306, -0.14368756115436554, -0.26660147309303284, -0.16515260934829712, -0.41453051567077637]), Row(id=70, features=[-0.5575206279754639, -0.37716078758239746, -0.15413062274456024, 0.5250346660614014, 0.22711136937141418, -0.08119841665029526, 0.23502472043037415, -0.37903010845184326, 0.1964838057756424, 0.2756272852420807]), Row(id=80, features=[-0.5214827060699463, -0.16647201776504517, 0.13192495703697205, 0.526060163974762, 1.0684044361114502, 0.573097825050354, 0.22665250301361084, 0.20131278038024902, 0.17668700218200684, -0.04834532365202904]), Row(id=90, features=[-0.19814075529575348, -0.3580942749977112, 0.12193574756383896, 0.4392879605293274, 0.5309962630271912, 0.4429631531238556, -0.1698579341173172, -0.18099184334278107, 0.27801862359046936, -0.7437546253204346]), Row(id=100, features=[-0.3665241003036499, -0.23866397142410278, 0.24534502625465393, 0.00041441884241066873, 0.11948510259389877, 0.20143750309944153, 0.3445690870285034, -0.2977309226989746, -0.10492483526468277, -0.15585187077522278]), Row(id=110, features=[-0.19042114913463593, -0.5498335957527161, 0.4264240264892578, -0.29253560304641724, 0.19845665991306305, -0.014012875035405159, -0.1418849676847458, -0.1245146319270134, 0.21188409626483917, -0.01412843819707632]), Row(id=120, features=[-0.3218458294868469, -0.4054754376411438, -0.02487514168024063, 0.019978510215878487, 0.019788088276982307, -0.4135362505912781, 1.0316457748413086, -0.0011786830145865679, -0.13891839981079102, -0.42483606934547424]), Row(id=130, features=[-0.7668135166168213, -0.6124680042266846, -0.5014193654060364, 1.442710280418396, 0.6719774603843689, 0.4149438142776489, 0.6010640263557434, 0.29646769165992737, 0.2412337213754654, -0.5126058459281921]), Row(id=140, features=[-0.15970438718795776, -0.3949264585971832, 0.012913763523101807, -0.28961554169654846, 0.43329480290412903, 0.11357028782367706, 0.29467344284057617, 0.2406676709651947, 0.0918106809258461, -0.35424935817718506]), Row(id=150, features=[-0.3439732789993286, -0.2759574353694916, 0.27033114433288574, -0.3490431308746338, 0.17510734498500824, 0.1763191819190979, -0.21238437294960022, -0.3173685073852539, 0.006783857475966215, -0.18565939366817474]), Row(id=160, features=[-0.6911455392837524, -0.11083552986383438, -0.0718071460723877, 0.06301499903202057, -0.10054247081279755, 0.23454929888248444, 0.280343621969223, -0.2719404399394989, 0.2808513939380646, -0.08190535753965378]), Row(id=170, features=[-0.5927299857139587, -0.238186314702034, -0.10026395320892334, 0.22045336663722992, 0.24740171432495117, -0.17572155594825745, 0.22629666328430176, -0.16480058431625366, 0.1711117923259735, -0.2855988144874573

In [0]:
print(list_items_factors[0])

Row(id=10, features=[-0.5054094791412354, -0.16811057925224304, 0.029730208218097687, -0.24545277655124664, 0.3562713861465454, -0.11089249700307846, 0.07500549405813217, -0.17772066593170166, 0.16993831098079681, 0.05939692631363869])

### Using Nearest neighbours

Pick a few movies, and for each of them, find-out the top 5 nearest neighbours. This is very similar to an optional question of the PLSA project...

In [0]:
movie_vectors_df=model.itemFactors.join(df.withColumnRenamed)
#on va faire la kn n avec une queury qui serit un vecteur

Make sur your KNN algorithm is fast enough. Try to understand why some results are not so good.